# [Memory](https://langchain-ai.github.io/langgraph/concepts/memory/)
- LangGraph의 Memory는 AI가 대화 내용을 기억할 수 있게 해주는 기능입니다. 
- 사람이 대화할 때 이전에 말한 내용을 기억하는 것처럼, AI도 과거 대화를 기억해서 더 자연스러운 대화를 할 수 있게 해줍니다.


## 왜 Memory가 필요한가요?
> Memory가 없다면:
```text
    사용자: "내 이름은 김철수야"
    AI: "안녕하세요!"

    사용자: "내 이름이 뭐였지?"
    AI: "죄송해요, 모르겠습니다"
```
> Memory가 있다면:
```text
    사용자: "내 이름은 김철수야"
    AI: "안녕하세요 김철수님!"

    사용자: "내 이름이 뭐였지?"
    AI: "김철수님이라고 하셨어요!"
```


# ConversationSummaryMemory 예시
**오래된 대화를 요약해서 기억하는 똑똑한 챗봇**

모든 대화를 기억하되, 오래된 내용은 요약해서 저장하는 방법입니다!


In [ ]:
# 필요한 라이브러리 import
from dotenv import load_dotenv

# 환경변수 로드
load_dotenv()

### 1. AI 모델 설정

In [21]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

### 2. 요약 기능이 포함된 노드 정의

In [22]:
from langchain_core.messages import SystemMessage

def summarize_old_messages(messages, max_messages=4):
    """오래된 메시지들을 요약하는 함수"""
    if len(messages) <= max_messages:
        return messages
    
    # 오래된 메시지와 최근 메시지 분리
    old_messages = messages[:-max_messages]
    recent_messages = messages[-max_messages:]
    
    # 요약 만들기
    summary_text = "이전 대화 요약:\\n"
    for msg in old_messages:
        if msg.type == "human":
            summary_text += f"- 사용자: {msg.content}\\n"
        elif msg.type == "ai":
            summary_text += f"- AI: {msg.content}\\n"
    
    # 요약을 시스템 메시지로 만들기
    summary_message = SystemMessage(content=summary_text)
    
    return [summary_message] + recent_messages

In [23]:
from langgraph.graph import MessagesState

def chat_node_with_summary(state: MessagesState):
    # 메시지가 너무 많으면 요약
    summarized_messages = summarize_old_messages(state["messages"])
    
    response = llm.invoke(summarized_messages)
    return {"messages": [response]}

### 3. 그래프 생성

In [24]:
from langgraph.graph import StateGraph, MessagesState, START, END 

graph_summary = StateGraph(MessagesState)
graph_summary.add_node("chat", chat_node_with_summary)
graph_summary.add_edge(START, "chat")
graph_summary.add_edge("chat", END)

### 4. 컴파일

In [25]:
from langgraph.checkpoint.memory import MemorySaver

memory_summary = MemorySaver()
app_with_summary = graph_summary.compile(checkpointer=memory_summary)

print("Summary Memory 챗봇이 준비되었습니다! (오래된 대화는 요약해서 기억)")

Summary Memory 챗봇이 준비되었습니다! (오래된 대화는 요약해서 기억)


### 5. 테스트

In [26]:
from langchain_core.messages import HumanMessage
import uuid

# 중요: 같은 thread_id를 사용해야 대화가 연결됩니다!
memory_id = str(uuid.uuid4())
# Summary Memory 테스트
config_summary = {"configurable": {"thread_id": memory_id}}

print("=== Summary Memory 챗봇 테스트 ===")
print("많은 정보를 말해보고, 나중에 요약된 정보로 기억하는지 확인해보겠습니다!\\n")

=== Summary Memory 챗봇 테스트 ===
많은 정보를 말해보고, 나중에 요약된 정보로 기억하는지 확인해보겠습니다!\n


In [27]:
# 테스트할 메시지들
test_messages = [
    "내 이름은 김철수야",
    "나는 서울에 살아", 
    "취미는 축구야",
    "좋아하는 음식은 피자야",
    "오늘 날씨가 좋네",
    "좋아하는 과일은 수박,복숭아야",
    "내 이름과 사는 곳이 뭐였지?"  # 오래된 정보가 요약되어 기억되는지 테스트
]

for i, msg in enumerate(test_messages, 1):
    print(f"{i}번째 대화: '{msg}'")
    result = app_with_summary.invoke(
        {"messages": [HumanMessage(content=msg)]}, 
        config=config_summary
    )
    print(f"AI: {result['messages'][-1].content}")
    print("-" * 40)

print("\\n결과: 오래된 대화는 요약되지만 중요한 정보는 기억합니다!")
print("메모리 효율적이면서도 핵심 정보를 유지해요!")

1번째 대화: '내 이름은 김철수야'
AI: 안녕하세요, 김철수님! 어떻게 도와드릴까요?
----------------------------------------
2번째 대화: '나는 서울에 살아'
AI: 서울에 사시는군요! 서울은 정말 다양한 문화와 매력이 있는 도시죠. 서울에서 좋아하는 장소나 활동이 있으신가요?
----------------------------------------
3번째 대화: '취미는 축구야'
AI: 축구를 좋아하시군요! 축구는 정말 재미있고 팀워크도 중요한 스포츠죠. 어떤 팀을 응원하시나요? 또는 자주 축구를 하시나요?
----------------------------------------
4번째 대화: '좋아하는 음식은 피자야'
AI: 피자를 좋아하시군요! 피자는 다양한 토핑과 스타일로 즐길 수 있어서 정말 매력적인 음식이죠. 어떤 종류의 피자를 가장 좋아하시나요? 마르게리타, 페퍼로니, 아니면 다른 특별한 토핑이 있나요?
----------------------------------------
5번째 대화: '오늘 날씨가 좋네'
AI: 날씨가 좋다니 기분이 좋으시겠어요! 이렇게 좋은 날에는 밖에 나가서 활동하기 좋은 것 같아요. 혹시 오늘 특별한 계획이 있으신가요?
----------------------------------------
6번째 대화: '좋아하는 과일은 수박,복숭아야'
AI: 수박과 복숭아를 좋아하시군요! 여름철에 특히 맛있는 과일들이죠. 수박은 시원하고 상큼해서 더위를 식히기에 좋고, 복숭아는 달콤하고 부드러워서 정말 맛있죠. 두 과일을 함께 먹으면 더 맛있을 것 같아요! 과일을 자주 드시나요?
----------------------------------------
7번째 대화: '내 이름과 사는 곳이 뭐였지?'
AI: 당신의 이름은 김철수님이고, 서울에 사신다고 하셨습니다!
----------------------------------------
\n결과: 오래된 대화는 요약되지만 중요한 정보는 기억합니다